In [ ]:
#%pip install pyyaml
#%pip install opentelemetry-api
#%pip install opentelemetry-sdk
#%pip install opentelemetry-instrumentation
#%pip install azure-mgmt-monitor
#%pip install azure.monitor.opentelemetry



## This sample demonstrates how to multiple agents using AgentTeam with traces.

In [ ]:
"""
USAGE:
    python sample_agents_agent_team_custom_team_leader.py

    Before running the sample:

    pip install azure-ai-agents azure-identity

    Set these environment variables with your own values:
    PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                       page of your Azure AI Foundry portal.
    MODEL_DEPLOYMENT_NAME - the name of the model deployment to use.
"""

import os
from typing import Optional, Set
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from utils.agent_team import AgentTeam, AgentTask
from utils.agent_trace_configurator import AgentTraceConfigurator
from azure.ai.agents.models import FunctionTool, ToolSet

from dotenv import load_dotenv
load_dotenv()

True

In [21]:
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

In [6]:
def create_task(team_name: str, recipient: str, request: str, requestor: str) -> str:
    """
    Requests another agent in the team to complete a task.

    :param team_name (str): The name of the team.
    :param recipient (str): The name of the agent that is being requested to complete the task.
    :param request (str): A description of the to complete. This can also be a question.
    :param requestor (str): The name of the agent who is requesting the task.
    :return: True if the task was successfully received, False otherwise.
    :rtype: str
    """
    task = AgentTask(recipient=recipient, task_description=request, requestor=requestor)
    team: Optional[AgentTeam] = None
    try:
        team = AgentTeam.get_team(team_name)
    except:
        pass
    if team is not None:
        team.add_task(task)
        return "True"
    return "False"


In [7]:
# Any additional functions that might be used by the agents:
agent_team_default_functions: Set = {
    create_task,
}

In [8]:
default_function_tool = FunctionTool(functions=agent_team_default_functions)

agents_client.enable_auto_function_calls({create_task})

if model_deployment_name is not None:
    AgentTraceConfigurator(agents_client=agents_client).setup_tracing()
    with agents_client:
        agent_team = AgentTeam("test_team", agents_client=agents_client)
        toolset = ToolSet()
        toolset.add(default_function_tool)
        agent_team.set_team_leader(
            model=model_deployment_name,
            name="TeamLeader",
            instructions="You are an agent named 'TeamLeader'. You are a leader of a team of agents. The name of your team is 'test_team'."
            "You are an agent that is responsible for receiving requests from user and utilizing a team of agents to complete the task. "
            "When you are passed a request, the only thing you will do is evaluate which team member should do which task next to complete the request. "
            "You will use the provided create_task function to create a task for the agent that is best suited for handling the task next. "
            "You will respond with the description of who you assigned the task and why. When you think that the original user request is "
            "processed completely utilizing all the talent available in the team, you do not have to create anymore tasks. "
            "Using the skills of all the team members when applicable is highly valued. "
            "Do not create parallel tasks. "
            "Here are the other agents in your team: "
            "- Coder: You are software engineer who writes great code. Your name is Coder. "
            "- Reviewer: You are software engineer who reviews code. Your name is Reviewer.",
            toolset=toolset,
        )
        agent_team.add_agent(
            model=model_deployment_name,
            name="Coder",
            instructions="You are software engineer who writes great code. Your name is Coder.",
        )
        agent_team.add_agent(
            model=model_deployment_name,
            name="Reviewer",
            instructions="You are software engineer who reviews code. Your name is Reviewer.",
        )
        agent_team.assemble_team()

        print("A team of agents specialized in software engineering is available for requests.")
        while True:
            user_input = input("Input (type 'quit' or 'exit' to exit): ")
            if user_input.lower() == "quit":
                break
            elif user_input.lower() == "exit":
                break
            agent_team.process_request(request=user_input)

        agent_team.dismantle_team()
else:
    print("Error: Please define the environment variable MODEL_DEPLOYMENT_NAME.")


Select a tracing option:
1. Enable Azure Monitor tracing
2. Enable console tracing without enabling gen_ai agent traces
3. Enable console tracing with gen_ai agent traces
4. Do not enable traces
No tracing enabled.
A team of agents specialized in software engineering is available for requests.
Created thread with ID: thread_hQU2VH122h1OTVItQyRKCu6K
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the _create_task function available for you to create the task. The request is: 
hello
'.
Created message with ID: msg_rwSNIEtXkYg11CWC2cp2VfNw for task in thread thread_hQU2VH122h1OTVItQyRKCu6K
Created and processed run for agent 'TeamLeader', run ID: run_ppv0jeWL3e6b5zpnoukiFVf3
Agent 'TeamLeader' completed task. Outcome: I have assigned the task to Coder, since the user said "hello" and it is a simple interaction. Coder is the most suitable agent to determine

##     This sample demonstrates how to use an AgentTeam to execute a multi-stepuser request with automatic function calling and trace collection.

In [17]:
"""
DESCRIPTION:

    The team consists of
        • one leader agent - created automatically from the configuration in
          `utils/agent_team_config.yaml`
        • three worker agents - `TimeWeatherAgent`, `SendEmailAgent`, and
          `TemperatureAgent`, each defined in the code below with its own tools

    IMPORTANT - leader-agent model configuration
        `utils/agent_team_config.yaml` contains the key TEAM_LEADER_MODEL.
        Its value must be the name of a **deployed** model in your Azure AI
        project (e.g. "gpt-4o-mini").
        If this deployment does not exist, AgentTeam cannot instantiate the
        leader agent and the sample will fail.

USAGE:
    python sample_agents_multi_agent_team.py

    Before running the sample:

    1. pip install azure-ai-agents azure-identity
    2. Ensure `utils/agent_team_config.yaml` is present and TEAM_LEADER_MODEL points
       to a valid model deployment.
    3. Set these environment variables with your own values:
         PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                            page of your Azure AI Foundry portal.
         MODEL_DEPLOYMENT_NAME - The model deployment name used for the worker agents.
"""

import os
from typing import Set

from utils.user_functions_with_traces import (
    fetch_current_datetime,
    fetch_weather,
    send_email_using_recipient_name,
    convert_temperature,
)

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential
from utils.agent_team import AgentTeam, _create_task
from utils.agent_trace_configurator import AgentTraceConfigurator
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

In [19]:
user_function_set_1: Set = {fetch_current_datetime, fetch_weather}

user_function_set_2: Set = {send_email_using_recipient_name}

user_function_set_3: Set = {convert_temperature}

agents_client.enable_auto_function_calls(
    {
        _create_task,
        fetch_current_datetime,
        fetch_weather,
        send_email_using_recipient_name,
        convert_temperature,
    }
)

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

In [20]:
if model_deployment_name is not None:
    AgentTraceConfigurator(agents_client=agents_client).setup_tracing()
    with agents_client:

        functions = FunctionTool(functions=user_function_set_1)
        toolset1 = ToolSet()
        toolset1.add(functions)

        agent_team = AgentTeam("test_team", agents_client=agents_client)

        agent_team.add_agent(
            model=model_deployment_name,
            name="TimeWeatherAgent",
            instructions="You are a specialized agent for time and weather queries.",
            toolset=toolset1,
            can_delegate=True,
        )

        functions = FunctionTool(functions=user_function_set_2)
        toolset2 = ToolSet()
        toolset2.add(functions)

        agent_team.add_agent(
            model=model_deployment_name,
            name="SendEmailAgent",
            instructions="You are a specialized agent for sending emails.",
            toolset=toolset2,
            can_delegate=False,
        )

        functions = FunctionTool(functions=user_function_set_3)
        toolset3 = ToolSet()
        toolset3.add(functions)

        agent_team.add_agent(
            model=model_deployment_name,
            name="TemperatureAgent",
            instructions="You are a specialized agent for temperature conversion.",
            toolset=toolset3,
            can_delegate=False,
        )

        agent_team.assemble_team()

        user_request = (
            "Hello, Please provide me current time in '%Y-%m-%d %H:%M:%S' format, and the weather in Toronto. "
            "Finally, convert the Celsius to Fahrenheit and send an email to Example Recipient with summary of results."
        )

        # Once process_request is called, the TeamLeader will coordinate.
        # The loop in process_request will pick up tasks from the queue, assign them, and so on.
        agent_team.process_request(request=user_request)

        agent_team.dismantle_team()
else:
    print("Error: Please define the environment variable MODEL_DEPLOYMENT_NAME.")


Select a tracing option:
1. Enable Azure Monitor tracing
2. Enable console tracing without enabling gen_ai agent traces
3. Enable console tracing with gen_ai agent traces
4. Do not enable traces


Overriding of current TracerProvider is not allowed


Console tracing enabled with agent traces.
Created thread with ID: thread_E4Ov2YSs6Q9xjweg1CCAT49y
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the _create_task function available for you to create the task. The request is: 
Hello, Please provide me current time in '%Y-%m-%d %H:%M:%S' format, and the weather in Toronto. Finally, convert the Celsius to Fahrenheit and send an email to Example Recipient with summary of results.
'.
Created message with ID: msg_i7KtJiRzdMsKDX5ALhMDDHFm for task in thread thread_E4Ov2YSs6Q9xjweg1CCAT49y
Created and processed run for agent 'TeamLeader', run ID: run_qlRGBh5AIjQ1GcvC2teBVg1A
Agent 'TeamLeader' completed task. Outcome: I have created tasks for TimeWeatherAgent to provide time and weather information and for TemperatureAgent to convert the temperature from Celsius to Fahrenheit once the temperature data is rece